In [1]:
import os
import utils.cv_reviewer_utils as cv_utils
from IPython.display import Markdown, display
import json

In [2]:
#if its published somewhere
#job_description = get_job_description("url")

job_description = """

Job Title: Junior Data Engineer
Location: Remote
Job Type: Full-time
Team: Data Engineering
Reports to: Senior Data Engineer

About the Role:

We are looking for a Junior Data Engineer to join our growing data team. You will support the design, development, and maintenance of data pipelines and infrastructure that power data analytics, reporting, and data-driven decision-making across the company.
This is a great opportunity for someone early in their career to learn and grow while working on real-world data challenges in a collaborative and agile environment.

Key Responsibilities
- Assist in building, maintaining, and optimizing ETL/ELT pipelines.
- Work closely with data analysts, scientists, and engineers to support data ingestion and transformation workflows.
- Help ensure the reliability and quality of data across systems.
- Monitor and troubleshoot data pipelines and processes.
- Contribute to documentation of data flows, models, and architecture.
- Learn and apply best practices in data engineering, including security and scalability.

Requirements:

- Basic knowledge of SQL and at least one programming language (Python preferred).
- Familiarity with data storage systems (e.g., relational databases, cloud storage).
- Understanding of data processing concepts and tools.
- Willingness to learn and grow in a fast-paced environment.
- Good communication and problem-solving skills.
- Bachelor’s degree in Computer Science, Engineering, Mathematics, or related field (or equivalent practical experience).

Nice to Have:

- Exposure to cloud platforms like AWS, GCP, or Azure.
- Experience with version control (e.g., Git).
- Basic knowledge of data modeling concepts.
- Familiarity with tools like Airflow, dbt, or Spark.

"""

In [3]:

# in lower case
mandatory_keywords = ["spark","python"]
language = "spanish"

landing_path = './landing'
        
# "llama3.2" or chatgpt model
model = "llama3.2"

In [4]:
analysis_text = cv_utils.analyze_candidates(model, job_description, mandatory_keywords, landing_path, language)
display(Markdown(analysis_text))

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


▶️ Using llama3.2 via Ollama...


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


▶️ Using llama3.2 via Ollama...


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


▶️ Using llama3.2 via Ollama...


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


▶️ Using llama3.2 via Ollama...


### 📊 Candidate Evaluations


**🧑‍💼 Candidate #1: alan susa.pdf – 80%**

Tiene experiencia en diseño y implementación de pipelines de datos en tiempo real utilizando Spark y PySpark. También tiene conocimientos en bases de datos NoSQL (MongoDB) y SQL (Postgres, Redshift, MySQL). Aunque es un Data Engineer Intern, su experiencia sugiere que podría ser capaz de asistir en la construcción y mantenimiento de pipelines de datos, pero podría necesitar orientación adicional.

**📝 Recommended Questions**
- ¿Cuál es tu nivel actual de experiencia como engineer de datos?
- ¿Cómo manejas el aprendizaje continuo en un entorno de tecnología en constante evolución?

---

**🧑‍💼 Candidate #2: john smith.pdf – 68%**

El candidato tiene experiencia en Spark y Python como tecnologías favoritas. También tiene conocimientos básicos de SQL y ha trabajado con ETL pipelines. Sin embargo, su senioridad no coincide con la del puesto, ya que busca un Junior Data Engineer.

**📝 Recommended Questions**
- ¿Cuál es tu nivel actual de experiencia en data engineering?
- Cómo crees que tus habilidades en ETL y SQL se adaptan al puesto de Junior Data Engineer?

---

**🧑‍💼 Candidate #3: Thaddeus Drake.pdf – 66%**

meets most of the requirements, lacking experience with Spark and cloud platforms, has strong knowledge on Python and database systems.

**📝 Recommended Questions**
- Can you explain your experience with data processing concepts and tools?
- How familiar are you with cloud platforms like AWS, GCP, or Azure?

---

**🧑‍💼 Candidate #4: Elara Quinn.pdf – 60%**

has some relevant skills like data visualization and relational database management system, but lacks direct experience with ETL/ELT pipelines, cloud platforms and tools like Airflow or dbt. Seniority is lower than expected for a Junior Data Engineer.

**📝 Recommended Questions**
- What experience do you have with ETL/ELT pipelines?
- Can you explain how you would optimize data ingestion and transformation workflows?
- How familiar are you with cloud platforms like AWS, GCP or Azure?

---
